In [32]:
import re
import os
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [33]:
def fetch(url):
    result = requests.get(url)
    soup = BeautifulSoup(result.content, "html.parser")
    return soup

In [34]:
def get_categories():
    top_category_url = "https://se.trustpilot.com/categories"
    category_soup = fetch(top_category_url)
    tags = category_soup.find("script", {"id":"__NEXT_DATA__"})
    data = json.loads(tags.text)
    tree = data["props"]
    all_categories = []
    cats = tree["pageProps"]["categories"]
    for category in cats:
        all_categories.append(category["categoryId"])
    category_urls = []
    for cat in all_categories:
        category_url = top_category_url + "/" + cat
        category_urls.append(category_url) # All category URLs collected from site
    return category_urls

In [35]:
categories = get_categories()

In [36]:
print(categories)

['https://se.trustpilot.com/categories/animals_pets', 'https://se.trustpilot.com/categories/electronics_technology', 'https://se.trustpilot.com/categories/events_entertainment', 'https://se.trustpilot.com/categories/vehicles_transportation', 'https://se.trustpilot.com/categories/business_services', 'https://se.trustpilot.com/categories/health_medical', 'https://se.trustpilot.com/categories/home_garden', 'https://se.trustpilot.com/categories/hobbies_crafts', 'https://se.trustpilot.com/categories/home_services', 'https://se.trustpilot.com/categories/legal_services_government', 'https://se.trustpilot.com/categories/construction_manufactoring', 'https://se.trustpilot.com/categories/food_beverages_tobacco', 'https://se.trustpilot.com/categories/media_publishing', 'https://se.trustpilot.com/categories/money_insurance', 'https://se.trustpilot.com/categories/travel_vacation', 'https://se.trustpilot.com/categories/restaurants_bars', 'https://se.trustpilot.com/categories/public_local_services', 

In [41]:
if not os.path.exists("data"):
    os.makedir("data")

for category in categories[0:1]:
    category_dir = category.split("/")
    category_dir = category_dir[len(category_dir)-1]
    if not os.path.exists("data/" + category_dir):
        os.mkdir("data/" + category_dir)
    
    category_specific_companies = []
    category_soup = fetch(category)
    category_tags = category_soup.find("script", {"id":"__NEXT_DATA__"})
    category_data = json.loads(category_tags.text)
    # Category is for example https://se.trustpilot.com/categories/animals_pets
    n_pages = category_data["props"]["pageProps"]["seoData"]["maxPages"]
    for i in range(1, n_pages):
        category_page = category + "?page=" + str(i)
        company_soup = fetch(category_page)
        company_tags = company_soup.find("script", {"id":"__NEXT_DATA__"})
        company_data = json.loads(company_tags.text)
        try: 
            
            #print(company_data)
            company_tree = company_data["props"]["pageProps"]["businessUnits"]
            for k, v in company_tree.items():
                if type(v) == list:
                    for j in v:
                        company_id = j["identifyingName"]
                        if company_id not in category_specific_companies:
                            category_specific_companies.append(company_id)
        except AttributeError:
            print("COMPANY ERROR! There was an error with this company:" + company_id)
        i += 1
    for c in category_specific_companies:
        if not os.path.exists("data/" + category_dir + "/" + c):
            os.mkdir("data/" + category_dir + "/" + c)
        count_soup = fetch("https://se.trustpilot.com/review/" + c)
        count_tags = count_soup.find("script", {"id":"__NEXT_DATA__"})
        count_data = json.loads(count_tags.text)
        count_tree = count_data["props"]
        count_pages = count_tree["pageProps"]["filters"]["pagination"]["totalPages"]
        for page in range(1, count_pages):
            try:
                review_soup = fetch("https://se.trustpilot.com/review/" + c + "?page=" + str(page))
                review_tags = review_soup.find("script", {"id":"__NEXT_DATA__"})
                review_data = json.loads(review_tags.text)
                review_tree = review_data["props"]["pageProps"]
                review_tree = review_tree["reviews"]
                review = review_tree[0]["text"]
                rating = review_tree[0]["rating"]
                print(rating, review, c, category)
            except AttributeError:
                print("PAGE ERROR! There was an error on this page: " + "https://se.trustpilot.com/review/" + c + "?page=" + str(page))

5 Trevlig och kunnig personal. Bra förslag som jag gillade direkt. Ingen tvekan utan jag köpte direkt.
Bra utbud och olika storlekar på täcken och halsband. eskilstunahundcenter.se https://se.trustpilot.com/categories/animals_pets
5 Tycker alla på Eskilstuna Hundcenter är toppen. Man får alltid bra hjälp av personalen, Annika är grym på hundmassage och Ingela är underbar att gå kurser hos. Fem av fem tassar! eskilstunahundcenter.se https://se.trustpilot.com/categories/animals_pets
5 Beställde en bok som nästan låg i brevlådan innan jag beställt färdigt. Toppen med snabb leverans eskilstunahundcenter.se https://se.trustpilot.com/categories/animals_pets
5 Snabb leverans, bra förpackat och bra produkter för ett bra pris. eskilstunahundcenter.se https://se.trustpilot.com/categories/animals_pets
5 Bra kommunikation. Bra bemötande. Snabb leverans eskilstunahundcenter.se https://se.trustpilot.com/categories/animals_pets
4 Jag tycker det var en bra valpkurs eskilstunahundcenter.se https://se.t

5 Enkel och bra hemsida, tydlig orderbekräftelse och leveransinformation.
Toppen, helt enkelt! hansesgarden.se https://se.trustpilot.com/categories/animals_pets
5 Snabb leverans!! hansesgarden.se https://se.trustpilot.com/categories/animals_pets
5 Jättebra! Täcker hela bilen. hansesgarden.se https://se.trustpilot.com/categories/animals_pets
5 Mycket snabb leverans! hansesgarden.se https://se.trustpilot.com/categories/animals_pets
5 Bra på alla sätt. Smidigt att beställa, snabb leverans. Härligt utbud av produkter. Jag återkommer helt klart. hansesgarden.se https://se.trustpilot.com/categories/animals_pets
5 Super service med hög kvalité på produkterna! hansesgarden.se https://se.trustpilot.com/categories/animals_pets
4 Det gick snabbt och jag fick vad jag betalade för. Jag är jättenöjd hansesgarden.se https://se.trustpilot.com/categories/animals_pets
5 Snabb leverans 🚚 och boken var väldigt fin. Tydliga bilder och beskrivningar hansesgarden.se https://se.trustpilot.com/categories/anima

5 Beställde en sele till hunden. Jättebra sele och leveransen gick smidigt och bra. hansesgarden.se https://se.trustpilot.com/categories/animals_pets
5 Mycket nöjd!
Enkelt att beställa. Enkel betalning. Snabb leverans! hansesgarden.se https://se.trustpilot.com/categories/animals_pets
4 Snabb och rätt leverans hansesgarden.se https://se.trustpilot.com/categories/animals_pets
5 Bäst när det gäller hansesgarden.se https://se.trustpilot.com/categories/animals_pets
5 Hansegården levererade snabbt och utmärkt. hansesgarden.se https://se.trustpilot.com/categories/animals_pets
4 Mycket bra kvalité på produkterna. hansesgarden.se https://se.trustpilot.com/categories/animals_pets
5 Snabb leverans. Mycket bra hansesgarden.se https://se.trustpilot.com/categories/animals_pets
5 Snabb leverans. Bra varor. hansesgarden.se https://se.trustpilot.com/categories/animals_pets
5 Kan verkligen rekommendera. Snabb service. Finns inget att klaga på hansesgarden.se https://se.trustpilot.com/categories/animals_

5 Snabb och bra service, varorna kom redan dagen efter beställningen. Dessutom hade varorna jag köpte mycket bättre pris här än på någon annan internetbutik. Mycket nöjd med mitt köp. hansesgarden.se https://se.trustpilot.com/categories/animals_pets
5 Trevligt bemötande i telefonen. Jag fick den information jag ville ha.
Och att få varan till dörren var ju ett plus också. Jag är nöjd. Hundburen
passade också perfekt. hansesgarden.se https://se.trustpilot.com/categories/animals_pets
5 Bra pris & snabb leverans. hansesgarden.se https://se.trustpilot.com/categories/animals_pets
5 Jätteenkelt att beställa och snabb leverans! hansesgarden.se https://se.trustpilot.com/categories/animals_pets
4 Varor som håller god kvalitet och passande pris hansesgarden.se https://se.trustpilot.com/categories/animals_pets
5 Enkelt och beställa, snabb leverans och enkelt att betala genom Klarna. hansesgarden.se https://se.trustpilot.com/categories/animals_pets
5 Enkelt att beställa och otroligt snabb leverans

5 Snabb leverans 4dogs.se https://se.trustpilot.com/categories/animals_pets
5 Snabb leverans o bra kvalitet 4dogs.se https://se.trustpilot.com/categories/animals_pets
5 Lätt att beställa, snabbt levererat 4dogs.se https://se.trustpilot.com/categories/animals_pets
5 Snabba leveranser..bra priser 4dogs.se https://se.trustpilot.com/categories/animals_pets
5 Bra produkter snabb leverans smidig betalning via Klarna 4dogs.se https://se.trustpilot.com/categories/animals_pets
5 Snabb leverans 4dogs.se https://se.trustpilot.com/categories/animals_pets
5 Alltid bra service och snabb leverans.
Är supernöjd 4dogs.se https://se.trustpilot.com/categories/animals_pets
5 Grym kvalitet. Snabb leverans.
Trevlig kundtjänst som kunde ändra vårt val av färg på trimbord från svart till blått då vi ändrat oss i sista sekund.
Helnöjda. 4dogs.se https://se.trustpilot.com/categories/animals_pets
5 Gick snabbt och bekvämt. 4dogs.se https://se.trustpilot.com/categories/animals_pets
5 Snabb leverans och bra kvalit

4 Jag är inte helt nöjd med kopplet som jag köpte. Det är inte mjukt och skönt utan det är stelt. Resten är jag nöjd med. www.propet.se https://se.trustpilot.com/categories/animals_pets
5 Snabb och exakt leverans ! Tack🙌🏻 www.propet.se https://se.trustpilot.com/categories/animals_pets
5 Fina läderkoppel i bra kvalitet www.propet.se https://se.trustpilot.com/categories/animals_pets
5 Kom snabbt o bra pris www.propet.se https://se.trustpilot.com/categories/animals_pets
5 Snabb handel. Men jag får se omLissi tycker om kylbädden hon är lite försiktig av sig www.propet.se https://se.trustpilot.com/categories/animals_pets
5 Alltid snabb leverans 😀 www.propet.se https://se.trustpilot.com/categories/animals_pets
5 Snabb leverans! www.propet.se https://se.trustpilot.com/categories/animals_pets
5 Bra mat. Goda tuggben. Jättesnabb leverans. SUPER www.propet.se https://se.trustpilot.com/categories/animals_pets
5 Alltid supersnabb frakt! www.propet.se https://se.trustpilot.com/categories/animals_pe

KeyboardInterrupt: 